In [ ]:
import torch
import os
import sys
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.insert(0, module_path)

import Learning.autoencoders as autoencoders
import Double_Pendulum.robot_parameters as robot_parameters
import Double_Pendulum.dynamics as dynamics
import Double_Pendulum.transforms as transforms

import Double_Pendulum.normal_form as normal_form

from datetime import datetime

rp = robot_parameters.LUMPED_PARAMETERS
rp["m1"] = 0.
model = autoencoders.Analytic_transformer(rp)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_cw = False

%load_ext autoreload
%autoreload 2

In [ ]:
def plot_single_vs_time(single_list, dt, ylabel, title, ylim = None, log = False):
    time = torch.linspace(0, (single_list.shape[0] - 1) * dt, single_list.shape[0]).numpy()
    single_np = single_list.cpu().numpy()

    plt.figure(figsize=(7, 3))
    plt.plot(time, single_np, label=ylabel)
    plt.xlabel("Time (s)")
    plt.ylabel(ylabel)
    plt.title(title)
    if ylim is not None:
        plt.ylim(ylim)
    if log:
        plt.yscale("log")
    plt.legend()
    plt.grid(True)
    #plt.yscale("log")
    plt.tight_layout()
    plt.show()

def plot_double_vs_time(double_list, dt, ylabel, title, ylim = None, log = False):
    time = torch.linspace(0, (double_list.shape[0] - 1) * dt, double_list.shape[0]).numpy()
    double_np = double_list.cpu().numpy()

    plt.figure(figsize=(7, 3))
    plt.plot(time, double_np[:, 0], label=ylabel+"0")
    plt.plot(time, double_np[:, 1], label=ylabel+"1")
    plt.xlabel("Time (s)")
    plt.ylabel(ylabel)
    plt.title(title)
    if ylim is not None:
        plt.ylim(ylim)
    if log:
        plt.yscale("log")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_quad_vs_time(quad_list, dt, ylabel, title, ylim = None, log = False):
    time = torch.linspace(0, (quad_list.shape[0] - 1) * dt, quad_list.shape[0]).numpy()
    quad_np = quad_list.cpu().numpy()
    

    plt.figure(figsize=(7, 3))
    plt.plot(time, quad_np[:, 0], label="Y")
    plt.plot(time, quad_np[:, 1], label="Y'")
    plt.plot(time, quad_np[:, 2], label="Y''")
    plt.plot(time, quad_np[:, 3], label="Y'''")
    plt.xlabel("Time (s)")
    plt.ylabel(ylabel)
    plt.title(title)
    if ylim is not None:
        plt.ylim(ylim)
    if log:
        plt.yscale("log")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
xy_start = torch.tensor([4., -1.9]).requires_grad_().to(device)
q_start = transforms.inverse_kinematics(xy_start, rp, is_clockwise=model_cw).unsqueeze(0)
q_d_start = torch.tensor([[0., 0.]]).requires_grad_().to(device)
q_dd_start = torch.tensor([[0., 0.]]).requires_grad_().to(device)


xy_des_real = torch.tensor([1.9262, -2.1004]).requires_grad_().to(device)
q_des = transforms.inverse_kinematics(xy_des_real, rp, is_clockwise=model_cw).unsqueeze(0)
q_d_des = torch.tensor([[0., 0.]]).requires_grad_().to(device)

In [ ]:
K = torch.tensor([[10., 100., 100., 100.]]).to(device)
k_spring = 0.5

In [ ]:
is_clockwise_start = transforms.check_clockwise(q_start.squeeze(0))


th_start = model.encoder_vmap(q_start)
th_d_start = (model.jacobian_enc(q_start) @ q_d_start.T).T

is_clockwise_des = transforms.check_clockwise(q_des.squeeze(0))

th_des = model.encoder_vmap(q_des)
th_d_des = (model.jacobian_enc(q_des) @ q_d_des.T).T

#th_des = torch.tensor([[6.1008,  1.558694425]]).requires_grad_().to(device)
#th_d_des = torch.tensor([[0.,  0.]]).requires_grad_().to(device)


q_des_hat = model.decoder_vmap(th_des, is_clockwise_des)
q_d_des_hat = (model.jacobian_dec(th_des, clockwise=is_clockwise_des) @ th_d_des.T).T
xy_des_est, _ = transforms.forward_kinematics(rp, q_des_hat[0])

print("th_start:", th_start)
print("th_d_start:", th_d_start)
print("th_des", th_des)
print("q_des_hat", q_des_hat)
print("xy_des_est", xy_des_est)

In [ ]:
J_h_inv_des = model.jacobian_dec(th_des, is_clockwise_des).squeeze(0)
J_h_inv_trans_des = torch.transpose(J_h_inv_des, 0, 1)

M_q_des, C_q_des, G_q_des = dynamics.dynamical_matrices(rp, q_des_hat.squeeze(0), q_d_des_hat.squeeze(0))
G_q_des = dynamics.add_spring_force_G_q(rp, q_des_hat, G_q_des, k_spring)
M_th_des, C_th_des, G_th_des = transforms.transform_dynamical_from_inverse(M_q_des, C_q_des, G_q_des, th_des, th_d_des, J_h_inv_des, J_h_inv_trans_des)


Y_des_u = normal_form.calculate_Y(th_des, th_d_des, M_th_des, G_th_des, device)
Y_des = torch.tensor([[Y_des_u[0,0]], [0], [0], [0]]).to(device)
print("Y_des_u:\n", Y_des_u)
print("Y_des:\n", Y_des)

### Sim in q-space

In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

th_series_qsim, th_d_series_qsim, th_dd_series_qsim = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
q_est_series_qsim, q_d_est_series_qsim, q_dd_est_series_qsim = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
q_real_series_qsim, q_d_real_series_qsim, q_dd_real_series_qsim = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
Y_series_qsim = torch.empty((0,4)).to(device)
u_series_qsim = torch.empty((0,1)).to(device)


dt = 0.01
t_end = 5
t_series = torch.arange(0, t_end, dt)

th = th_start
th_d = th_d_start

th_thsim = th_start
th_d_thsim = th_d_start

q_est = model.decoder(th_start, clockwise=is_clockwise_start)
q_d_est = (model.jacobian_dec(th_start, clockwise=is_clockwise_start) @ th_d_start.T).T
q_dd_est = torch.zeros((1,2))

q_real, q_d_real, q_dd_real = q_start, q_d_start, q_dd_start

is_clockwise = transforms.check_clockwise(q_start.squeeze(0))

for t in torch.arange(0, t_end, dt):
    t_string = "Time: [" + str(t.item().__round__(3)) + "/" + str(t_end) + ".0]"
    print(t_string)

    q = q_real
    q_d = q_d_real

    th = model.encoder_vmap(q)
    th_d = (model.jacobian_enc(q) @ q_d.T).T

    q_hat = model.decoder_vmap(th, clockwise=model_cw)
    q_d_hat = (model.jacobian_dec(th) @ th_d.T).T
    
    """ Obtain Jacobian, dynamical matrices"""

    is_clockwise = transforms.check_clockwise(q_hat.squeeze(0))
    
    J_h_inv = model.jacobian_dec(th, is_clockwise).squeeze(0)
    J_h_inv_trans = torch.transpose(J_h_inv, 0, 1)

    M_q_est, C_q_est, G_q_est = dynamics.dynamical_matrices(rp, q_hat.squeeze(0), q_d_hat.squeeze(0))
    G_q_est = dynamics.add_spring_force_G_q(rp, q_hat, G_q_est, k_spring)
    A_q_est = dynamics.input_matrix(rp, q_hat.squeeze(0))


    """ Feed-forward simulation of the system, not on real dynamics """
    M_th, C_th, G_th = transforms.transform_dynamical_from_inverse(M_q_est, C_q_est, G_q_est, th, th_d, J_h_inv, J_h_inv_trans)
    M_th = torch.tensor([[rp["m2"], 0.], [0., rp["m2"]]]).to(device).requires_grad_(True)
    M_th = M_th * th/th
    C_th = torch.tensor([[0, 0.], [0., 0.]]).to(device).requires_grad_(True)
    A_th = transforms.transform_input_matrix_from_inverse_trans(A_q_est, J_h_inv_trans)

    Y = normal_form.calculate_Y(th, th_d, M_th, G_th, device)
    alpha, beta = normal_form.calculate_alpha_beta(th, th_d, M_th, G_th, A_th, Y)

    #print("alpha:", alpha)
    #print("beta:", beta)
    v = normal_form.calculate_v(Y, Y_des, K)
    u = normal_form.calculate_u(alpha, beta, v)
    #print("Y:", Y)
    #print("v:", v)
    #print("u:", u)

    """ Recalculate "real system" based on modified matrices in theta-space. """

    J_h = model.jacobian_enc(q_hat)
    J_h_trans = torch.transpose(J_h, 0, 1)

    """ Update the real system and apply latent control input. """

    M_q_real, C_q_real, G_q_real = transforms.transform_dynamical_from_inverse(M_th, C_th, G_th, q_hat, q_d_hat, J_h, J_h_trans)
    A_q_real = transforms.transform_input_matrix_from_inverse_trans(A_th, J_h_trans)    

    tau_q_real = A_q_real * u
    q_dd_real = (torch.pinverse(M_q_real) @ (tau_q_real - C_q_real @ ((q_d_real).T)- G_q_real)).T 
    q_d_real = q_d_real + q_dd_real * dt
    q_real = q_real + q_d_real * dt
    q_real = transforms.wrap_to_pi(q_real)
    

    q_real_shifted = transforms.shift_q(q_real, clockwise=model_cw)
    th = model.encoder_vmap(q_real_shifted)
    q_est = model.decoder_vmap(th, clockwise=transforms.check_clockwise(q_real.squeeze(0)))
    q_est = transforms.wrap_to_pi(q_est)
    th_d = (model.jacobian_enc(q_real) @ q_d_real.T).T
    q_d_est = (model.jacobian_dec(th, clockwise=is_clockwise) @ th_d.T).T
    q_dd_est = torch.tensor([[0., 0.]]).requires_grad_().to(device) #TODO: Implement with Jacobian derivative


    """ Store data for plotting """

    th_series_qsim = torch.cat((th_series_qsim, th.detach()), dim=0)
    th_d_series_qsim = torch.cat((th_d_series_qsim, th_d.detach()), dim=0)
    th_dd_series_qsim = torch.cat((th_dd_series_qsim, th_dd.detach()), dim=0)
    
    q_real_series_qsim = torch.cat((q_real_series_qsim, q_real.detach()), dim=0)
    q_d_real_series_qsim = torch.cat((q_d_real_series_qsim, q_d_real.detach()), dim=0)
    q_dd_real_series_qsim = torch.cat((q_dd_real_series_qsim, q_dd_real.detach()), dim=0)

    q_est_series_qsim = torch.cat((q_est_series_qsim, q_est.detach()), dim=0)
    q_d_est_series_qsim = torch.cat((q_d_est_series_qsim, q_d_est.detach()), dim=0)
    q_dd_est_series_qsim = torch.cat((q_dd_est_series_qsim, q_dd_est.detach()), dim=0) 

    u_series_qsim = torch.cat((u_series_qsim, u.detach()), dim=0)
    Y_series_qsim = torch.cat((Y_series_qsim, Y.detach().T))
    print("")

    if torch.isnan(q[0,0]):
        break

    
    


### Sim in $\theta$-space